# 使用助手API（GPT-4）和DALL·E-3创建幻灯片

本笔记本演示了如何利用新的[助手API](https://platform.openai.com/docs/assistants/overview)（GPT-4）和DALL·E-3来制作信息丰富且视觉吸引人的幻灯片。<br>
创建幻灯片是许多工作的关键方面，但可能是费力且耗时的。此外，从数据中提取见解并有效地表达在幻灯片上可能具有挑战性。<br><br>本食谱将演示如何利用新的助手API来为您简化端到端的幻灯片创建过程，而无需触碰Microsoft PowerPoint或Google幻灯片，节省您宝贵的时间和精力！


## 0. 设置


In [1]:
from IPython.display import display, Image
from openai import OpenAI
import os
import pandas as pd
import json
import io
from PIL import Image
import requests

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "<your OpenAI API key if not set as env var>"))

# 让我们从 https://cookbook.openai.com/examples/assistants_api_overview_python 导入一些助手辅助函数
def show_json(obj):
    display(json.loads(obj.model_dump_json()))

def submit_message(assistant_id, thread, user_message,file_ids=None):
    params = {
        'thread_id': thread.id,
        'role': 'user',
        'content': user_message,
    }
    if file_ids:
        params['file_ids']=file_ids

    client.beta.threads.messages.create(
        **params
)
    return client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
)

def get_response(thread):
    return client.beta.threads.messages.list(thread_id=thread.id)



## 1. 创建内容


在这个示例中，我们将为我们公司NotReal Corporation的季度财务审查创建一个简短的虚构演示文稿。我们希望突出一些影响公司盈利能力的关键趋势。<br>假设我们有一些财务数据可供使用。让我们加载数据，然后看一下...


In [2]:
financial_data_path = 'data/NotRealCorp_financial_data.json'
financial_data = pd.read_json(financial_data_path)
financial_data.head(5)



,Year,Quarter,Distribution channel,Revenue ($M),Costs ($M),Customer count,Time
0,2021,Q1,Online Sales,1.50,1.301953,150,2021 Q1
1,2021,Q1,Direct Sales,1.50,1.380809,151,2021 Q1
2,2021,Q1,Retail Partners,1.50,1.348246,152,2021 Q1
3,2021,Q2,Online Sales,1.52,1.308608,152,2021 Q2
4,2021,Q2,Direct Sales,1.52,1.413305,153,2021 Q2


正如您所看到的，这些数据包括不同分销渠道的季度收入、成本和客户数据。让我们创建一个助手，可以充当个人分析师，并为我们的PowerPoint制作出漂亮的可视化图表！


首先，我们需要上传我们的文件，这样我们的助手才能访问它。


In [3]:
file = client.files.create(
  file=open('data/NotRealCorp_financial_data.json',"rb"),
  purpose='assistants',
)



现在，我们已经准备好创建我们的助手了。我们可以指示我们的助手扮演数据科学家的角色，接受我们给出的任何查询，并运行必要的代码来输出正确的数据可视化。这里的指令参数类似于ChatCompletions端点中的系统指令，可以帮助指导助手。我们还可以打开代码解释器工具，这样我们的助手就能编写代码了。最后，我们可以指定我们想要使用的任何文件，这种情况下只有我们上面创建的`financial_data`文件。


In [4]:
assistant = client.beta.assistants.create(
  instructions="You are a data scientist assistant. When given data and a query, write the proper code and create the proper visualization",
  model="gpt-4-1106-preview",
  tools=[{"type": "code_interpreter"}],
  file_ids=[file.id]
)



现在让我们创建一个线程，作为我们的第一个请求，要求助手计算季度利润，然后按分销渠道随时间变化绘制利润图。助手将自动计算每个季度的利润，并创建一个新列，将季度和年份合并在一起，而无需我们直接要求。我们还可以指定每条线的颜色。


In [5]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "Calculate profit (revenue minus cost) by quarter and year, and visualize as a line plot across the distribution channels, where the colors of the lines are green, light red, and light blue",
      "file_ids": [file.id]
    }
  ]
)



现在我们可以执行线程的运行。


In [6]:

run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)



现在我们可以开始一个循环，检查图像是否已经创建。注意：这可能需要几分钟。


In [7]:
messages = client.beta.threads.messages.list(thread_id=thread.id)



In [8]:
import time

while True:
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    try:
        #检查图像是否已创建
        messages.data[0].content[0].image_file
        #睡眠以确保运行已完成
        time.sleep(5)
        print('Plot created!')
        break
    except:
        time.sleep(10)
        print('Assistant still working...')



Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Plot created!


让我们看看助手添加的消息。


In [9]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
[message.content[0] for message in messages.data]



[MessageContentImageFile(image_file=ImageFile(file_id='file-0rKABLygI02MgwwhpgWdRFY1'), type='image_file'),
 MessageContentText(text=Text(annotations=[], value="The profit has been calculated for each distribution channel by quarter and year. Next, I'll create a line plot to visualize these profits. As specified, I will use green for the 'Online Sales', light red for 'Direct Sales', and light blue for 'Retail Partners' channels. Let's create the plot."), type='text'),
 MessageContentText(text=Text(annotations=[], value="The JSON data has been successfully restructured into a tabular dataframe format. It includes the year, quarter, distribution channel, revenue, costs, customer count, and a combined 'Time' representation of 'Year Quarter'. Now, we have the necessary information to calculate the profit (revenue minus cost) by quarter and year.\n\nTo visualize the profit across the different distribution channels with a line plot, we will proceed with the following steps:\n\n1. Calculate 

我们可以看到助手的最后一条消息（最新消息首先显示）包含了我们正在寻找的图像文件。这里有一个有趣的地方是，助手能够尝试多次解析JSON数据，因为第一次解析失败了，展示了助手的适应能力。


In [10]:
# 快速辅助函数，用于将我们的输出文件转换为PNG格式。
def convert_file_to_png(file_id, write_path):
    data = client.files.content(file_id)
    data_bytes = data.read()
    with open(write_path, "wb") as file:
        file.write(data_bytes)



In [11]:
plot_file_id = messages.data[0].content[0].image_file.file_id
image_path = "../images/NotRealCorp_chart.png"
convert_file_to_png(plot_file_id,image_path)

#上传
plot_file = client.files.create(
  file=open(image_path, "rb"),
  purpose='assistants'
)



让我们加载图表！


![图片](../images/NotRealCorp_chart.png)


很好！所以，只用一句话，我们就能让助手使用代码解释器计算盈利能力，并绘制各种分销渠道的三条线图。现在我们有了漂亮的可视化图表，但我们希望还有一些见解与之相配。


## 2. 生成洞察力


要从我们的图像中获得见解，我们只需要在我们的对话中添加一条新消息。我们的助手会利用消息历史为我们提供一些简明的观点。


In [12]:
submit_message(assistant.id,thread,"Give me two medium length sentences (~20-30 words per sentence) of the \
      most important insights from the plot you just created.\
             These will be used for a slide deck, and they should be about the\
                     'so what' behind the data."
)



Run(id='run_NWoygMcBfHUr58fCE4Cn6rxN', assistant_id='asst_3T362kLlTyAq0FUnkvjjQczO', cancelled_at=None, completed_at=None, created_at=1701827074, expires_at=1701827674, failed_at=None, file_ids=['file-piTokyHGllwGITzIpoG8dok3'], instructions='You are a data scientist assistant. When given data and a query, write the proper code and create the proper visualization', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_73TgtFoJMlEJvb13ngjTnAo3', tools=[ToolAssistantToolsCode(type='code_interpreter')])

现在，一旦运行完成，我们就可以看到最新的消息。


In [13]:
# 由于助手可能会对要点进行迭代，因此需要硬编码等待响应。
time.sleep(10)
response = get_response(thread)
bullet_points = response.data[0].content[0].text.value
print(bullet_points)



The plot reveals a consistent upward trend in profits for all distribution channels, indicating successful business growth over time. Particularly, 'Online Sales' shows a notable increase, underscoring the importance of digital presence in revenue generation.


太棒了！我们的助手能够识别出在线销售利润的显著增长，并推断这显示了拥有强大的数字化存在的重要性。现在让我们为幻灯片找一个引人注目的标题。


In [14]:
submit_message(assistant.id,thread,"Given the plot and bullet points you created,\
 come up with a very brief title for a slide. It should reflect just the main insights you came up with."
)



Run(id='run_q6E85J31jCw3QkHpjJKl969P', assistant_id='asst_3T362kLlTyAq0FUnkvjjQczO', cancelled_at=None, completed_at=None, created_at=1701827084, expires_at=1701827684, failed_at=None, file_ids=['file-piTokyHGllwGITzIpoG8dok3'], instructions='You are a data scientist assistant. When given data and a query, write the proper code and create the proper visualization', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_73TgtFoJMlEJvb13ngjTnAo3', tools=[ToolAssistantToolsCode(type='code_interpreter')])

标题是：


In [15]:
#等待助手可能需要几步操作
time.sleep(10)
response = get_response(thread)
title = response.data[0].content[0].text.value
print(title)



"Ascending Profits & Digital Dominance"


## 3. DALL·E-3 标题图像


很好，现在我们有一个标题、一个情节和两个项目符号。我们几乎可以把这一切放在幻灯片上了，但作为最后一步，让DALL·E-3为演示文稿的标题幻灯片设计一幅图片。<br><br>
*注意：* DALL·E-3目前还不在助手API中可用，但即将推出！<br><br>
我们将提供我们公司（NotRealCorp）的简要描述，让DALL·E-3来完成剩下的工作！


In [16]:
company_summary = "NotReal Corp is a prominent hardware company that manufactures and sells processors, graphics cards and other essential computer hardware."



In [17]:
response = client.images.generate(
  model='dall-e-3',
  prompt=f"given this company summary {company_summary}, create an inspirational \
    photo showing the growth and path forward. This will be used at a quarterly\
       financial planning meeting",
       size="1024x1024",
       quality="hd",
       n=1
)
image_url = response.data[0].url



好的，现在我们可以将这张图片添加到我们的线程中。首先，我们可以将图片保存在本地，然后使用`File`上传端点将其上传到助手API。让我们也看一下我们的图片。


In [18]:
dalle_img_path = '../images/dalle_image.png'
img = requests.get(image_url)

#本地保存
with open(dalle_img_path,'wb') as file:
  file.write(img.content)

#上传
dalle_file = client.files.create(
  file=open(dalle_img_path, "rb"),
  purpose='assistants'
)



  
![图片](../images/dalle_image.png)


## 4. 创建幻灯片


我们现在已经有了创建幻灯片所需的所有内容。虽然我们可以简单地添加一条消息请求幻灯片，但让我们改为给助手提供一个幻灯片模板，使用`python-pptx`库。这样可以确保我们得到我们想要的风格的幻灯片。请查看笔记本末尾的`扩展`部分，了解如何创建模板的说明。


In [19]:
title_template = """
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_PARAGRAPH_ALIGNMENT
from pptx.dml.color import RGBColor

# Create a new presentation object
prs = Presentation()

# Add a blank slide layout
blank_slide_layout = prs.slide_layouts[6]
slide = prs.slides.add_slide(blank_slide_layout)

# Set the background color of the slide to black
background = slide.background
fill = background.fill
fill.solid()
fill.fore_color.rgb = RGBColor(0, 0, 0)

# Add image to the left side of the slide with a margin at the top and bottom
left = Inches(0)
top = Inches(0)
height = prs.slide_height
width = prs.slide_width * 3/5
pic = slide.shapes.add_picture(image_path, left, top, width=width, height=height)

# Add title text box positioned higher
left = prs.slide_width * 3/5
top = Inches(2)
width = prs.slide_width * 2/5
height = Inches(1)
title_box = slide.shapes.add_textbox(left, top, width, height)
title_frame = title_box.text_frame
title_p = title_frame.add_paragraph()
title_p.text = title_text
title_p.font.bold = True
title_p.font.size = Pt(38)
title_p.font.color.rgb = RGBColor(255, 255, 255)
title_p.alignment = PP_PARAGRAPH_ALIGNMENT.CENTER

# Add subtitle text box
left = prs.slide_width * 3/5
top = Inches(3)
width = prs.slide_width * 2/5
height = Inches(1)
subtitle_box = slide.shapes.add_textbox(left, top, width, height)
subtitle_frame = subtitle_box.text_frame
subtitle_p = subtitle_frame.add_paragraph()
subtitle_p.text = subtitle_text
subtitle_p.font.size = Pt(22)
subtitle_p.font.color.rgb = RGBColor(255, 255, 255)
subtitle_p.alignment = PP_PARAGRAPH_ALIGNMENT.CENTER
"""

data_vis_template = """
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_PARAGRAPH_ALIGNMENT
from pptx.dml.color import RGBColor

# Create a new presentation object
prs = Presentation()

# Add a blank slide layout
blank_slide_layout = prs.slide_layouts[6]
slide = prs.slides.add_slide(blank_slide_layout)

# Set the background color of the slide to black
background = slide.background
fill = background.fill
fill.solid()
fill.fore_color.rgb = RGBColor(0, 0, 0)

# Define placeholders
image_path = data_vis_img
title_text = "Maximizing Profits: The Dominance of Online Sales & Direct Sales Optimization"
bullet_points = "• Online Sales consistently lead in profitability across quarters, indicating a strong digital market presence.\n• Direct Sales show fluctuations, suggesting variable performance and the need for targeted improvements in that channel."

# Add image placeholder on the left side of the slide
left = Inches(0.2)
top = Inches(1.8)
height = prs.slide_height - Inches(3)
width = prs.slide_width * 3/5
pic = slide.shapes.add_picture(image_path, left, top, width=width, height=height)

# Add title text spanning the whole width
left = Inches(0)
top = Inches(0)
width = prs.slide_width
height = Inches(1)
title_box = slide.shapes.add_textbox(left, top, width, height)
title_frame = title_box.text_frame
title_frame.margin_top = Inches(0.1)
title_p = title_frame.add_paragraph()
title_p.text = title_text
title_p.font.bold = True
title_p.font.size = Pt(28)
title_p.font.color.rgb = RGBColor(255, 255, 255)
title_p.alignment = PP_PARAGRAPH_ALIGNMENT.CENTER

# Add hardcoded "Key Insights" text and bullet points
left = prs.slide_width * 2/3
top = Inches(1.5)
width = prs.slide_width * 1/3
height = Inches(4.5)
insights_box = slide.shapes.add_textbox(left, top, width, height)
insights_frame = insights_box.text_frame
insights_p = insights_frame.add_paragraph()
insights_p.text = "Key Insights:"
insights_p.font.bold = True
insights_p.font.size = Pt(24)
insights_p.font.color.rgb = RGBColor(0, 128, 100)
insights_p.alignment = PP_PARAGRAPH_ALIGNMENT.LEFT
insights_frame.add_paragraph()


bullet_p = insights_frame.add_paragraph()
bullet_p.text = bullet_points
bullet_p.font.size = Pt(12)
bullet_p.font.color.rgb = RGBColor(255, 255, 255)
bullet_p.line_spacing = 1.5
"""




让我们为幻灯片设置一些快速变量。我们希望公司名称为NotRealCorp显示在标题幻灯片上，演示文稿的标题应为“2023年第三季度季度财务规划会议”。


In [20]:
title_text = "NotRealCorp"
subtitle_text = "Quarterly financial planning meeting, Q3 2023"



数据幻灯片中，我们有：


这里有一个创建标题幻灯片的模板。下面的模板是通过将理想的标题幻灯片图像上传到GPT-V，并请求生成用于创建该模板的`python-pptx`代码而创建的。模板的输入是image_path、title_text和subtitle_text。


In [21]:
submit_message(assistant.id,thread,f"Use the included code template to create a PPTX slide that follows the template format, but uses the image, company name/title, and document name/subtitle included:\
{title_template}. IMPORTANT: Use the image file included in this message as the image_path image in this first slide, and use the Company Name {title_text} as the title_text variable, and \
  use the subtitle_text {subtitle_text} a the subtitle_text variable. \
    NEST, create a SECOND slide using the following code template: {data_vis_template} to create a PPTX slide that follows the template format, but uses the company name/title, and document name/subtitle included:\
{data_vis_template}. IMPORTANT: Use the line plot image, that is the second attached image in this message, that you created earlier in the thread as the data_vis_img image, and use the data visualization title that you created earlier for the variable title_text, and\
  the bullet points of insights you created earlier for the bullet_points variable. Output these TWO SLIDES as a .pptx file. Make sure the output is two slides, with each slide matching the respective template given in this message.",
              file_ids=[dalle_file.id, plot_file.id]
)



Run(id='run_taLrnOnlDhoywgQFFBOLPlg0', assistant_id='asst_3T362kLlTyAq0FUnkvjjQczO', cancelled_at=None, completed_at=None, created_at=1701827118, expires_at=1701827718, failed_at=None, file_ids=['file-piTokyHGllwGITzIpoG8dok3'], instructions='You are a data scientist assistant. When given data and a query, write the proper code and create the proper visualization', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_73TgtFoJMlEJvb13ngjTnAo3', tools=[ToolAssistantToolsCode(type='code_interpreter')])

In [22]:
#可能需要1-3分钟
while True:
    try:
        response = get_response(thread)
        pptx_id = response.data[0].content[0].text.annotations[0].file_path.file_id
        print("Successfully retrieved pptx_id:", pptx_id)
        break
    except Exception as e:
        print("Assistant still working on PPTX...")
        time.sleep(10)



Assistant still working on PPTX...
Assistant still working on PPTX...
Assistant still working on PPTX...
Assistant still working on PPTX...
Assistant still working on PPTX...
Assistant still working on PPTX...
Assistant still working on PPTX...
Assistant still working on PPTX...
Assistant still working on PPTX...
Assistant still working on PPTX...
Successfully retrieved pptx_id: file-oa0i63qPH4IaJXYj90aA6L4Q


In [25]:
pptx_id = response.data[0].content[0].text.annotations[0].file_path.file_id
ppt_file= client.files.content(pptx_id)
file_obj = io.BytesIO(ppt_file.read())
with open("data/created_slides.pptx", "wb") as f:
    f.write(file_obj.getbuffer())



现在，我们已经保存了一个包含所有创建内容的PPTX文件！<br>


让我们仅使用助手API和DALL·E-3创建的.pptx文件的屏幕截图。在助手API中，我们还没有`seed`参数，因此DALL·E-3的图像和文字与您运行此笔记本时看到的略有不同，这是由于LLM的非确定性，但输出应该在方向上是相同的。


标题幻灯片:


![标题页](../images/title_slide.png)


数据幻灯片：


![数据幻灯片](../images/data_vis_slide.png)


## 5. 结论


哇！虽然这些幻灯片可能需要一些格式调整，但我们已经使用助手API、GPT-4和DALL·E-3制作了一些很棒的内容。我们能够使用助手来计算跨分销渠道的季度利润，绘制结果，从可视化中识别见解和关键要点，并创建一个总结性标题，所有这些都是基于一个包含财务数据的`.csv`文件。而且，仅凭借我们公司NotRealCorp的描述，我们使用DALL·E-3制作了一个令人印象深刻的标题图片。<br><br>
虽然我们距离完全在没有人为参与的情况下自动化这个过程还有一段路要走，但希望这个笔记本可以让您的幻灯片制作过程变得更加轻松一些。更重要的是，这个笔记本可以让您初步了解助手API的潜力！我们很期待看到您会构建出什么。


## 6. 扩展


- 当DALL·E-3被整合到助手API中时，我们将能够在线程中请求生成的标题图像。
- GPT-4-Vision目前尚不受助手API支持，但可以用于从线图像中收集见解。
- GPT-4-Vision被用于生成此示例中包含的`python-pptx`模板，因此一个潜在的扩展项目可以是演示将图像转换为幻灯片模板的最佳实践。
